# modules_import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import datetime
import random
import xarray as xr
import cartopy.crs as ccrs
import netCDF4
import matplotlib.pylab as pylab
import matplotlib.colors as mcolors
import h5py
import argparse
import pymap3d as pm
import os
import random
import time
import math
import torch
import copy

from scipy import signal
from torch.nn import Linear
from torch import Tensor
from torch.nn import MSELoss
from torch.optim import SGD, Adam, RMSprop
from torch.autograd import Variable, grad
from torch.cuda.amp import autocast
from torch.utils.data.sampler import SubsetRandomSampler,WeightedRandomSampler
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from pathlib import Path
from pyproj import Proj
from scipy.ndimage.filters import gaussian_filter
from glob import glob

# for reproducibility
os.environ['PYTHONHASHSEED']= '123'
np.random.seed(123)

params = {
    'xtick.labelsize':'xx-small',
    'ytick.labelsize':'xx-small',
    'figure.dpi':300
}
pylab.rcParams.update(params)

# model_parameters

## V(x,y,z)

In [4]:
# model specifications
ear_rad = 6371
v0 = 5
zmin = 0.; zmax = 2.; deltaz = 0.05;
xmin = 0.; xmax = 2.; deltax = 0.05;
ymin = 0.; ymax = 2.; deltay = 0.05;

# coordinates setup
z = np.arange(zmin,zmax+deltaz,deltaz)
nz = z.size
y = np.arange(ymin,ymax+deltay,deltay)
ny = y.size
x = np.arange(xmin,xmax+deltax,deltax)
nx = x.size
X,Y,Z = np.meshgrid(x,y,z)

# point-source location
sz = 1.; sx = 1.; sy=0.;
sx = x.flat[np.abs(x - sx).argmin()]
sy = y.flat[np.abs(y - sy).argmin()]
sz = z.flat[np.abs(z - sz).argmin()]
print(sx,sy,sz)

# hyperparameters
num_lay = 128*128
num_epo = int(2e3)
lea_rat = 5e-3
act_fun = 'relu'
num_pts = x.size
bat_siz = 4000
vel_typ = 'inhomogeneous'
vel_sha = 'sphere'
coo_sys = 'cartesian'
ker_ini = 'glorot_uniform'
res_net = True
opt_fun = 'adam'

# preparing velocity model
dv_dz = 0.5; dv_dx = 0.; dv_dy = 0.; 
vel_sou = 2. + dv_dz*sz + dv_dx*sx + dv_dy*sy
vel_mod = vel_sou + dv_dz*(Z-sz) + dv_dx*(X-sx) + dv_dy*(Y-sy)

# traveltime solution
if dv_dz==0 and dv_dx==0: 
    
  # for homogeneous velocity model
  T_data = np.sqrt((Z-sz)**2 + (Y-sy)**2 + (X-sx)**2)/2.
else: 
    
  # for velocity gradient model
  T_data = np.arccosh(1.0+0.5*(1.0/vel_mod)*(1/vel_sou)*(dv_dz**2 + dv_dy**2 + dv_dx**2)*((X-sx)**2 + (Y-sy)**2 + (Z-sz)**2))/np.sqrt(dv_dz**2 + dv_dy**2 + dv_dx**2)

# grid points for prediction 
X_star = [X.reshape(-1,1), ] 

# velocity at the source location
vel = vel_mod[int(round(sz/deltaz)),int(round(sx/deltax))]

# for plotting only
x_plot,y_plot,z_plot = X*ear_rad/1000,Y*ear_rad/1000,Z*ear_rad/1000

1.0 0.0 1.0


## V(rad,the,phi)

In [ ]:
# import
from projection import *

# model specifications
ear_rad = 6371
lat_sou = latitude.flat[np.abs(latitude - -8).argmin()]
lon_sou = longitude.flat[np.abs(longitude - 110).argmin()]
dep_sou = depth.flat[np.abs(depth - 1750).argmin()]

# coordinates setup
sx, sy, sz = pm.geodetic2ecef(lat_sou, lon_sou, -1e3*dep_sou)

# rescale
x,y,z = x/(ear_rad*1e3), y/(ear_rad*1e3), z/(ear_rad*1e3)
sx, sy, sz = sx/(ear_rad*1e3), sy/(ear_rad*1e3), sz/(ear_rad*1e3)

X,Y,Z = x,y,z

print(np.where((np.isclose(x.reshape(-1,1), sx)) & (np.isclose(y.reshape(-1,1), sy)) & (np.isclose(z.reshape(-1,1), sz))))
print(sx,sy,sz)

# for plotting only
x_plot,y_plot,z_plot = x.reshape(-1,1)*ear_rad/1000, y.reshape(-1,1)*ear_rad/1000, z.reshape(-1,1)*ear_rad/1000

# saving parameters
num_lay = 128*64
num_epo = int(2e3)
lea_rat = 1e-4
act_fun = 'tanh'
num_pts = x.size
bat_siz = num_pts//500
vel_typ = 'radial'
vel_sha = 'sphere'
coo_sys = 'cartesian'
ker_ini = 'glorot_uniform'
opt_fun = 'adam'
hyp_par = (
    str(num_lay) + '_' +
    str(num_epo) + '_' +
    str(lea_rat) + '_' +
    str(dep_dim) + '_' +
    str(lat_dim) + '_' +
    str(lon_dim) + '_' +
    act_fun + '_' +
    str(num_pts) + '_' +
    str(bat_siz) + '_' +
    vel_typ + '_' +
    vel_sha + '_' +
    coo_sys + '_' +
    ker_ini + '_' +
    opt_fun
)

# path
model_path = "./" + hyp_par
figures_path = model_path + '/'
checkpoints_path = figures_path + 'checkpoints' + '/'

Path(figures_path).mkdir(parents=True, exist_ok=True)
Path(checkpoints_path).mkdir(parents=True, exist_ok=True)

# model

In [15]:
# def init_weights(m):
#     if type(m) == torch.nn.Linear:
#         stdv = (1. / math.sqrt(m.weight.size(1))/1.)*2
#         m.weight.data.uniform_(-stdv,stdv)
#         m.bias.data.uniform_(-stdv,stdv)

def init_weights(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.xavier_normal(m.weight)

class NN(torch.nn.Module):
    def __init__(self, nl=10, activation=torch.nn.ELU()):
            super(NN, self).__init__()
            self.act = activation

            # Input Structure
            self.fc0  = Linear(2*3,32)
            self.fc1  = Linear(32,512)

            # Resnet Block
            self.rn_fc1 = torch.nn.ModuleList([Linear(512, 512) for i in range(nl)])
            self.rn_fc2 = torch.nn.ModuleList([Linear(512, 512) for i in range(nl)])
            self.rn_fc3 = torch.nn.ModuleList([Linear(512, 512) for i in range(nl)])

            # Output structure
            self.fc8  = Linear(512,32)
            self.fc9  = Linear(32,1)

    def forward(self,x):
        x   = self.act(self.fc0(x))
        x   = self.act(self.fc1(x))
        for ii in range(len(self.rn_fc1)):
            x0 = x
            x  = self.act(self.rn_fc1[ii](x))
            x  = self.act(self.rn_fc3[ii](x)+self.rn_fc2[ii](x0))

        x     = self.act(self.fc8(x))
        tau   = abs(self.fc9(x))
        return tau

# class NN(torch.nn.Module):
#     def __init__(self, nl=128, activation=torch.nn.ELU()):
#             super(NN, self).__init__()
#             self.act = activation

#             # Input Structure
#             self.fc0  = Linear(2*3,128)

#             # Hidden Layer
#             self.fc1  = torch.nn.ModuleList([Linear(128, 128) for i in range(nl)])

#             # Output structure
#             self.fc9  = Linear(128,1)

#     def forward(self,x):
#         x   = self.act(self.fc0(x))

#         for ii in range(len(self.fc1)):
#             x  = self.act(self.fc1[ii](x))

#         tau   = abs(self.fc9(x))
#         return tau

def EikonalLoss(Yobs,Xp,tau,device):
        dtau  = torch.autograd.grad(outputs=tau, inputs=Xp, grad_outputs=torch.ones(tau.size()).to(device), only_inputs=True,create_graph=True,retain_graph=True)[0]

        T0    = torch.sqrt(((Xp[:,3]-Xp[:,0])**2 + (Xp[:,4]-Xp[:,1])**2 + (Xp[:,5]-Xp[:,2])**2))  
        T1    = (T0**2)*(dtau[:,3]**2 + dtau[:,4]**2 + dtau[:,5]**2)
        T2    = 2*tau[:,0]*(dtau[:,3]*(Xp[:,3]-Xp[:,0]) + dtau[:,4]*(Xp[:,4]-Xp[:,1]) + dtau[:,5]*(Xp[:,5]-Xp[:,2]))
        T3    = tau[:,0]**2
        S2    = (T1+T2+T3)
        Ypred = torch.sqrt(1/S2)
        diff  = abs(Yobs[:,1]-Ypred)/Yobs[:,1]
        loss  = torch.mean(abs((Yobs[:,1]-Ypred)/Yobs[:,1]))
        return loss, diff


class Model():
    def __init__(self, ModelPath, VelocityClass, device='gpu'):
        
        self.Params = {}
        self.Params['ModelPath']     = ModelPath
        self.Params['VelocityClass'] = VelocityClass #Pass the JSON information
        self.Params['Device']        = device
        self.Params['Pytorch Amp (bool)'] = False

        self.Params['Network']  = {}
        self.Params['Network']['Number of Residual Blocks']  = 20
        self.Params['Network']['Layer activation']           = torch.nn.ELU()
        self.Params['Network']['Normlisation']               = 'MinMax'

        self.Params['Training'] = {}
        # self.Params['Training']['Number of sample points']   = 1e4
        self.Params['Training']['Batch Size']                = 64
        self.Params['Training']['Validation Percentage']     = 10
        self.Params['Training']['Number of Epochs']          = 200
        self.Params['Training']['Resampling Bounds']         = [0.1,0.9]
        self.Params['Training']['Print Every * Epoch']       = 1
        self.Params['Training']['Save Every * Epoch']        = 50
        self.Params['Training']['Learning Rate']             = 5e-5
        self.Params['Training']['Random Distance Sampling']  = False
        self.Params['Training']['Use Scheduler (bool)']      = True

        # Parameters to alter during training
        self.total_train_loss = []
        self.total_val_loss   = []


    def _init_network(self):
        self.network = NN(nl=self.Params['Network']['Number of Residual Blocks'],activation=self.Params['Network']['Layer activation'])
        self.network.apply(init_weights)
        self.network.float()
        self.network.to(torch.device(self.Params['Device']))



    def _projection(self,Xp,inverse=False):
        if type(self.Params['VelocityClass'].projection) != type(None):
            proj = Proj(self.Params['VelocityClass'].projection)
            Xp = Xp.detach().cpu().numpy()
            Xp[:,0],Xp[:,1] = proj(Xp[:,0],Xp[:,1],inverse=inverse)
            Xp[:,3],Xp[:,4] = proj(Xp[:,3],Xp[:,4],inverse=inverse)
            Xp = torch.Tensor(Xp)
            Xp = Xp.to(torch.device(self.Params['Device']))
        return Xp


    def  _normalization(self,Xp=None,Yp=None):

        # Loading the predefined variables
        if self.Params['Network']['Normlisation'] == 'MinMax':
            xmin_UTM = np.array(copy.copy(self.Params['VelocityClass'].xmin))
            xmax_UTM = np.array(copy.copy(self.Params['VelocityClass'].xmax))
            if type(self.Params['VelocityClass'].projection) == str:
                proj = Proj(self.Params['VelocityClass'].projection)
                xmin_UTM[0],xmin_UTM[1] = proj(xmin_UTM[0],xmin_UTM[1]) 
                xmax_UTM[0],xmax_UTM[1] = proj(xmax_UTM[0],xmax_UTM[1]) 
            indx = np.argmax(xmax_UTM-xmin_UTM)
            self.nf_max    = xmax_UTM[indx]
            self.nf_min    = xmin_UTM[indx]
            self.sf        = (self.nf_max-self.nf_min)

            if (type(Xp)!=type(None)) and (type(Yp)==type(None)):
                Xp  = Xp/self.sf
                return Xp
            if (type(Xp)==type(None)) and (type(Yp)!=type(None)):
                Yp  = Yp*self.sf
                return Yp
            else:
                Xp = Xp/self.sf
                Yp = Yp/self.sf
                return Xp,Yp

        if self.Params['Network']['Normlisation'] == 'OffsetMinMax':
            xmin_UTM = np.array(copy.copy(self.Params['VelocityClass'].xmin))
            xmax_UTM = np.array(copy.copy(self.Params['VelocityClass'].xmax))
            if type(self.Params['VelocityClass'].projection) == str:
                proj = Proj(self.Params['VelocityClass'].projection)
                xmin_UTM[0],xmin_UTM[1] = proj(xmin_UTM[0],xmin_UTM[1]) 
                xmax_UTM[0],xmax_UTM[1] = proj(xmax_UTM[0],xmax_UTM[1]) 
            indx = np.argmax(xmax_UTM-xmin_UTM)
            self.nf_max    = xmax_UTM[indx]
            self.nf_min    = xmin_UTM[indx]
            self.sf        = (self.nf_max-self.nf_min)

            self.crt_point = (xmax_UTM - xmin_UTM)/2 + xmin_UTM

            if (type(Xp)!=type(None)) and (type(Yp)==type(None)):
                for ii in [0,1,2]:
                    Xp[:,ii]   = Xp[:,ii]   - self.crt_point[ii]
                    Xp[:,ii+3] = Xp[:,ii+3] - self.crt_point[ii]
                Xp = (Xp)/self.sf
                return Xp
            if (type(Xp)==type(None)) and (type(Yp)!=type(None)):
                Yp  = Yp*self.sf
                return Yp
            else:
                for ii in [0,1,2]:
                    Xp[:,ii]   = Xp[:,ii]   - self.crt_point[ii]
                    Xp[:,ii+3] = Xp[:,ii+3] - self.crt_point[ii]
                Xp = (Xp)/self.sf
                Yp = (Yp)/self.sf
                return Xp,Yp

    def train(self):

        # Initialising the network
        self._init_network()

        # Defining the optimization scheme
        self.optimizer  = torch.optim.Adam(self.network.parameters(),lr=self.Params['Training']['Learning Rate'])
        if self.Params['Training']['Use Scheduler (bool)'] == True:
            self.scheduler  = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer)

        # Creating a sampling dataset
        self.dataset = Database(
            self.Params['ModelPath'],
            self.Params['VelocityClass'],
            create=True,
            Numsamples=int(self.Params['Training']['Number of sample points']),
            randomDist=self.Params['Training']['Random Distance Sampling']
        )
        self.dataset.send_device(torch.device(self.Params['Device']))
        self.dataset.data,self.dataset.target = self._normalization(Xp=self.dataset.data,Yp=self.dataset.target)

        len_dataset         = len(self.dataset)
        n_batches           = int(len(self.dataset)/int(self.Params['Training']['Batch Size']) + 1)
        training_start_time = time.time()

        # Splitting the dataset into training and validation
        indices            = list(range(int(len_dataset)))
        validation_idx     = np.random.choice(indices, size=int(len_dataset*(self.Params['Training']['Validation Percentage']/100)), replace=False)
        train_idx          = list(set(indices) - set(validation_idx))
        validation_sampler = SubsetRandomSampler(validation_idx)
        train_sampler      = SubsetRandomSampler(train_idx)

        train_loader       = torch.utils.data.DataLoader(
            self.dataset,
            batch_size=int(self.Params['Training']['Batch Size'] ),
            sampler=train_sampler,
            )    
        validation_loader  = torch.utils.data.DataLoader(
            self.dataset,
            batch_size=int(self.Params['Training']['Batch Size'] ),
            sampler=validation_sampler,
        )    

        # Defining the initial weights to sample by
        weights = Tensor(torch.ones(len(self.dataset))).to(torch.device(self.Params['Device']))
        weights[validation_idx] = 0.0
        print(weights.device)

        for epoch in range(1,self.Params['Training']['Number of Epochs']+1):
            print_every           = 1
            start_time            = time.time()
            running_sample_count  = 0
            total_train_loss      = 0
            total_val_loss        = 0

            # Defining the weighting of the samples
            weights                 = torch.clamp(weights/weights.max(),self.Params['Training']['Resampling Bounds'][0],self.Params['Training']['Resampling Bounds'][1])
            weights[validation_idx] = 0.0
            train_sampler_wei       = WeightedRandomSampler(weights, len(weights), replacement=True)
            train_loader_wei        = torch.utils.data.DataLoader(
                                        self.dataset,
                                        batch_size=int(self.Params['Training']['Batch Size'] ),
                                        sampler=train_sampler_wei,
                                      )
            weights                 = Tensor(torch.zeros(len(self.dataset))).to(torch.device(self.Params['Device']))

            for i, data in enumerate(train_loader_wei, 0):
                
                # Get inputs/outputs and wrap in variable object
                inputs, labels, indexbatch = data
                inputs = inputs.float()
                labels = labels.float()

                inputs.requires_grad_()


                if self.Params['Pytorch Amp (bool)']:
                    with autocast():
                        outputs = self.network(inputs)
                        loss_value, wv  = EikonalLoss(labels,inputs,outputs,torch.device(self.Params['Device']))
                else:
                    outputs = self.network(inputs)
                    loss_value, wv  = EikonalLoss(labels,inputs,outputs,torch.device(self.Params['Device']))


                loss_value.backward()

                # Update parameters
                self.optimizer.step()
                self.optimizer.zero_grad()

                # Updating the weights
                weights[indexbatch] = wv

                total_train_loss += loss_value.item()
                del inputs, labels, indexbatch, outputs, loss_value, wv


            # Determining the Training Loss
            for i, data_val in enumerate(validation_loader, 0):
                inputs_val, labels_val, indexbatch_val = data_val
                inputs_val = inputs_val.float()
                labels_val = labels_val.float()
                inputs_val.requires_grad_()

                if self.Params['Pytorch Amp (bool)']:
                    with autocast():
                        outputs_val                 = self.network(inputs_val)
                        val_loss,wv                 = EikonalLoss(labels_val,inputs_val,outputs_val,torch.device(self.Params['Device']))
                else:
                    outputs_val  = self.network(inputs_val)
                    val_loss,wv  = EikonalLoss(labels_val,inputs_val,outputs_val,torch.device(self.Params['Device']))

                total_val_loss             += val_loss.item()
                del inputs_val, labels_val, indexbatch_val, outputs_val, val_loss, wv


            # Creating a running loss for both training and validation data
            total_val_loss   /= len(validation_loader)
            total_train_loss /= len(train_loader)
            self.total_train_loss.append(total_train_loss)
            self.total_val_loss.append(total_val_loss)

            if self.Params['Training']['Use Scheduler (bool)'] == True:
                self.scheduler.step(total_val_loss)

            del train_loader_wei,train_sampler_wei

            if epoch % self.Params['Training']['Print Every * Epoch'] == 0:
                with torch.no_grad():
                    print("Epoch = {} -- Training loss = {:.4e} -- Validation loss = {:.4e}".format(epoch,total_train_loss,total_val_loss))

            if (epoch % self.Params['Training']['Save Every * Epoch'] == 0) or (epoch == self.Params['Training']['Number of Epochs'] ) or (epoch == 1):
                with torch.no_grad():
                    self.save(epoch=epoch,val_loss=total_val_loss)

    def save(self,epoch='',val_loss=''):
        torch.save(
            {
                'epoch':epoch,
                'model_state_dict': self.network.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'train_loss': self.total_train_loss,
                'val_loss': self.total_val_loss
            }, 
            '{}/Model_Epoch_{}_ValLoss_{}.pt'.format(self.Params['ModelPath'],str(epoch).zfill(5),val_loss)
        )

    def load(self,filepath):
        # -- Loading model information
        self._init_network()
        checkpoint            = torch.load(filepath,map_location=torch.device(self.Params['Device']))
        self.total_train_loss = checkpoint['train_loss']
        self.total_val_loss   = checkpoint['val_loss']
        self.network.load_state_dict(checkpoint['model_state_dict'])
        self.network.to(torch.device(self.Params['Device']))


    def TravelTimes(self,Xp,projection=True,normlisation=True):
        # Apply projection from LatLong to UTM
        Xp  = Xp.to(torch.device(self.Params['Device']))
        if projection:
            Xp  = self._projection(Xp)
        if normlisation:
            Xp  = self._normalization(Xp=Xp,Yp=None)
        tau = self.network(Xp)
        T0  = torch.sqrt(((Xp[:,3]-Xp[:,0])**2 + (Xp[:,4]-Xp[:,1])**2 + (Xp[:,5]-Xp[:,2])**2))
        TT  = tau[:,0]*T0
        del Xp,tau,T0
        return TT

    def Velocity(self,Xp,projection=True,normlisation=True):
        Xp    = Xp.to(torch.device(self.Params['Device']))
        if projection:
            Xp  = self._projection(Xp)
        if normlisation:
            Xp  = self._normalization(Xp=Xp,Yp=None)
        Xp.requires_grad_()
        tau   = self.network(Xp)
        dtau  = torch.autograd.grad(outputs=tau, inputs=Xp, grad_outputs=torch.ones(tau.size()).to(torch.device(self.Params['Device'])), 
                                    only_inputs=True,create_graph=True,retain_graph=True)[0]        
        T0    = torch.sqrt(((Xp[:,3]-Xp[:,0])**2 + (Xp[:,4]-Xp[:,1])**2 + (Xp[:,5]-Xp[:,2])**2))  
        T1    = (T0**2)*(dtau[:,3]**2 + dtau[:,4]**2 + dtau[:,5]**2)
        T2    = 2*tau[:,0]*(dtau[:,3]*(Xp[:,3]-Xp[:,0]) + dtau[:,4]*(Xp[:,4]-Xp[:,1]) + dtau[:,5]*(Xp[:,5]-Xp[:,2]))
        T3    = tau[:,0]**2
        Ypred = torch.sqrt(1/(T1+T2+T3))
        Ypred = self._normalization(Yp=Ypred)
        del Xp,tau,dtau,T0,T1,T2,T3
        return Ypred

class _numpy2dataset(torch.utils.data.Dataset):
    def __init__(self, data, target, transform=None):
        # Creating identical pairs
        self.data    = Variable(Tensor(data))
        self.target  = Variable(Tensor(target))

    def send_device(self,device):
        self.data    = self.data.to(device)
        self.target  = self.target.to(device)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        return x, y, index
    def __len__(self):
        return self.data.shape[0]

def _randPoints(numsamples=10000,randomDist=False,Xmin=[0,0,0],Xmax=[2,2,2]):
    numsamples = int(numsamples)
    Xmin = np.append(Xmin,Xmin)
    Xmax = np.append(Xmax,Xmax)
    if randomDist:
        X  = np.zeros((numsamples,6))
        PointsOutside = np.arange(numsamples)
        while len(PointsOutside) > 0:
            P  = np.random.rand(len(PointsOutside),3)*(Xmax[:3]-Xmin[:3])[None,None,:] + Xmin[:3][None,None,:]
            dP = np.random.rand(len(PointsOutside),3)-0.5
            rL = (np.random.rand(len(PointsOutside),1))*np.sqrt(np.sum((Xmax-Xmin)**2))
            nP = P + (dP/np.sqrt(np.sum(dP**2,axis=1))[:,np.newaxis])*rL

            X[PointsOutside,:3] = P
            X[PointsOutside,3:] = nP

            maxs          = np.any((X[:,3:] > Xmax[:3][None,:]),axis=1)
            mins          = np.any((X[:,3:] < Xmin[:3][None,:]),axis=1)
            OutOfDomain   = np.any(np.concatenate((maxs[:,None],mins[:,None]),axis=1),axis=1)
            PointsOutside = np.where(OutOfDomain)[0]
    else:
        X  = (np.random.rand(numsamples,6)*(Xmax-Xmin)[None,None,:] + Xmin[None,None,:])[0,:,:]
    return X


def Database(PATH,VelocityFunction,create=False,Numsamples=5000,randomDist=False,SurfaceRecievers=False):
    if create == True:
        xmin = copy.copy(VelocityFunction.xmin)
        xmax = copy.copy(VelocityFunction.xmax)

        # Projecting from LatLong to UTM
        if type(VelocityFunction.projection) == str:
            proj = Proj(VelocityFunction.projection)
            xmin[0],xmin[1] = proj(xmin[0],xmin[1])
            xmax[0],xmax[1] = proj(xmax[0],xmax[1])

        Xp   = _randPoints(numsamples=Numsamples,Xmin=xmin,Xmax=xmax,randomDist=randomDist)
        Yp   = VelocityFunction.eval(Xp)

        # Handling NaNs values
        while len(np.where(np.isnan(Yp[:,1]))[0]) > 0:
            indx     = np.where(np.isnan(Yp[:,1]))[0]
            print('Recomputing for {} points with nans'.format(len(indx)))
            Xpi      = _randPoints(numsamples=len(indx),Xmin=xmin,Xmax=xmax,randomDist=randomDist)
            Yp[indx,:] = VelocityFunction.eval(Xpi)
            Xp[indx,:] = Xpi

        # Saving the training dataset
        np.save('{}/Xp'.format(PATH),Xp)
        np.save('{}/Yp'.format(PATH),Yp)
    else:
        try:
            Xp = np.load('{}/Xp.npy'.format(PATH))
            Yp = np.load('{}/Yp.npy'.format(PATH))
        except ValueError:
            print('Please specify a correct source path, or create a dataset')


    print(Xp.shape,Yp.shape)
    database = _numpy2dataset(Xp,Yp)

    return database

class ToyProblem_Checkerboard:
    def __init__(self):
        self.xmin     = [0,0,0]
        self.xmax     = [20.,20.,20.]

        # projection 
        self.projection = None

        # Velocity values
        self.velocity_mean     = 5.0
        self.velocity_phase    = 0.5
        self.velocity_offset   = -2.5 
        self.velcoity_amp      = 1.0

    def eval(self,Xp):
        Yp = np.ones((Xp.shape[0],2))
        SinS = (signal.square(Xp[:,0]+self.velocity_offset,self.velocity_phase) + signal.square(Xp[:,1]+self.velocity_offset,self.velocity_phase) + signal.square(Xp[:,2]+self.velocity_offset,self.velocity_phase))/3
        SinR = (signal.square(Xp[:,3]+self.velocity_offset,self.velocity_phase) + signal.square(Xp[:,4]+self.velocity_offset,self.velocity_phase) + signal.square(Xp[:,5]+self.velocity_offset,self.velocity_phase))/3
        Yp[:,0] = (SinS)*self.velcoity_amp + self.velocity_mean
        Yp[:,1] = (SinR)*self.velcoity_amp + self.velocity_mean
        return Yp

In [16]:
filePath = './'
model = Model(filePath,VelocityClass=ToyProblem_Checkerboard(),device='cuda:0')
model.train()

/home/taufikmh/miniconda3/envs/thesis-pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  if __name__ == '__main__':


(10000, 6) (10000, 2)
cuda:0
Epoch = 1 -- Training loss = 1.5567e-01 -- Validation loss = 9.5890e-02
Epoch = 2 -- Training loss = 1.0995e-01 -- Validation loss = 9.8864e-02
Epoch = 3 -- Training loss = 1.4561e-01 -- Validation loss = 1.1124e-01
Epoch = 4 -- Training loss = 1.4514e-01 -- Validation loss = 9.7096e-02
Epoch = 5 -- Training loss = 1.5494e-01 -- Validation loss = 1.1705e-01
Epoch = 6 -- Training loss = 1.4851e-01 -- Validation loss = 1.3032e-01
Epoch = 7 -- Training loss = 1.4755e-01 -- Validation loss = 9.5921e-02
Epoch = 8 -- Training loss = 1.4881e-01 -- Validation loss = 9.8792e-02
Epoch = 9 -- Training loss = 1.5046e-01 -- Validation loss = 1.4586e-01
Epoch = 10 -- Training loss = 1.4075e-01 -- Validation loss = 1.3226e-01
Epoch = 11 -- Training loss = 1.4839e-01 -- Validation loss = 9.9825e-02
Epoch = 12 -- Training loss = 1.4717e-01 -- Validation loss = 9.9187e-02
Epoch = 13 -- Training loss = 1.3946e-01 -- Validation loss = 9.8860e-02
Epoch = 14 -- Training loss = 1

KeyboardInterrupt: 

# velocity_function

In [ ]:
def velocity_function(vel_sha='sphere', vel_typ='homogeneous', vel_ini=5, x=X, y=Y, z=Z):
    if vel_sha=='sphere':
        # inhomogeneous velocity without gaps
        vel_inh = vel_ini - (x**2 + z**2 + y**2)
        vel_inh[vel_inh<2.**2] = np.NaN
        if vel_typ=='inhomogeneous':  
            # velocity without gaps
            vel_all = np.copy(vel_inh)
            
            # velocity with gaps
            vel_gap = np.copy(vel_all)
            vel_gap[vel_inh>2.1**2] = np.NaN
        elif vel_typ == 'random':
            # velocity without gaps
            from scipy.ndimage import gaussian_filter
            vel_all = gaussian_filter(18*np.random.random((nx,ny,nz))**3, sigma=6)
            vel_all[np.isnan(vel_inh)] = np.NaN

            # velocity with gaps
            vel_gap = np.copy(vel_all)
            vel_gap[vel_inh>2.1**2] = np.NaN
        elif vel_typ == 'homogeneous':
            # velocity without gaps
            vel_all = vel_ini*np.ones_like(vel_inh)
            vel_all[np.isnan(vel_inh)] = np.NaN
            
            # velocity with gaps
            vel_tmp = np.copy(vel_inh)
            vel_tmp[vel_inh>2.1**2] = np.NaN
            vel_gap = np.copy(vel_all)
            vel_gap[np.isnan(vel_tmp)] = np.NaN
        elif vel_typ == 'radial':
            from generate_data import vp_gap
            vel_inh = vp
            vel_all = vp
            vel_gap = vp_gap
            
            
    return vel_inh, vel_all, vel_gap

# call function
vel_inh, vel_all, vel_gap = velocity_function(vel_typ=vel_typ)

# training_points

## V(x,y,z)

In [12]:
# define receiver coordinates
xR, yR, zR = X.reshape(-1,1), Y.reshape(-1,1), Z.reshape(-1,1)

# define source coordinates
xS, yS, zS = sx*np.ones_like(X.reshape(-1,1)), sy*np.ones_like(X.reshape(-1,1)), sz*np.ones_like(X.reshape(-1,1))

# define inputs and output
Xp = np.hstack((xS, yS, zS, xR, yR, zR))
yp = vel_mod.reshape(-1,1)

# random sampling
X_train, X_test, y_train, y_test = train_test_split(
    Xp[np.logical_not(np.isnan(yp))[:,0]], 
    yp[np.logical_not(np.isnan(yp))].reshape(-1,1), 
    test_size=0.33,
    random_state=125
)

# TensorFlow data pipeline
dat_tra = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dat_tra = dat_tra.batch(X_train.shape[0]+1)
dat_val = tf.data.Dataset.from_tensor_slices((X_test, y_test))
dat_val = dat_val.batch(X_test.shape[0]+1)                       
    
X_starf = [X_train[:,3].reshape(-1,1), X_train[:,4].reshape(-1,1), X_train[:,5].reshape(-1,1)]

# find source location id in X_star
TOL = 1e-11
sids,_ = np.where((np.isclose(X_starf[0], sx)) & (np.isclose(X_starf[1], sy)) & (np.isclose(X_starf[2], sz)))

print(sids)
print(sids.shape)
print(X_starf[0][sids,0])
print(X_starf[1][sids,0])
print(X_starf[2][sids,0])
print(sx,sy,sz)

[43834]
(1,)
[1.]
[0.]
[1.]
1.0 0.0 1.0


## V(rad,the,phi)

In [ ]:
# define receiver coordinates
xR, yR, zR = X.reshape(-1,1), Y.reshape(-1,1), Z.reshape(-1,1)

# define source coordinates
xS, yS, zS = sx*np.ones_like(X.reshape(-1,1)), sy*np.ones_like(X.reshape(-1,1)), sz*np.ones_like(X.reshape(-1,1))

# define inputs and output
Xp = np.hstack((xS, yS, zS, xR, yR, zR))
yp = vel_gap.reshape(-1,1)

# random sampling
X_train, X_test, y_train, y_test = train_test_split(
    Xp[np.logical_not(np.isnan(yp))[:,0]], 
    yp[np.logical_not(np.isnan(yp))].reshape(-1,1),
    test_size=0.3,
    random_state=1235
)

# find source location id in X_star
sids,_ = np.where(
    (np.isclose(X_train[:,3].reshape(-1,1), sx)) & 
    (np.isclose(X_train[:,4].reshape(-1,1), sy)) & 
    (np.isclose(X_train[:,5].reshape(-1,1), sz))
)

print(sids)
print(sx,sy,sz)

# TensorFlow data pipeline
dat_tra = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dat_tra = dat_tra.batch(bat_siz)
dat_val = tf.data.Dataset.from_tensor_slices((X_test, y_test))
dat_val = dat_val.batch(bat_siz) 

# train

In [ ]:
strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
with strategy.scope():
    
    # custom model compilation
    inp_eik = tf.keras.layers.Input(shape=(6,))
    for_net = feedforward_model.build()
    gra_lay = backpropagation_layer(for_net)
    out_eik = custom_ouput(gra_lay, inp_eik)

    # compile model
    cus_mod = custom_model([inp_eik], [out_eik])
    cus_mod.compile(optimizer="adam", metrics=["mse"])
    
    # callbacks
    pocket = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        min_delta=0.001,
        patience=5, 
        verbose=1,
        restore_best_weights = True
    )

    history = cus_mod.fit(
        dat_tra,
        epochs=3,
        callbacks=[pocket],
        validation_data=dat_val,
        workers=3, 
        use_multiprocessing=True
    )

# custom_training

In [14]:
# build a core tau model
tau = feedforward_model.build()

# build a PINN model
model = PINNs_model(tau).build()
model.summary()

lea_sch = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=400,
    decay_rate=0.90
)
optimizer = keras.optimizers.SGD(learning_rate=lea_sch)


optimizer = tf.keras.optimizers.Adam(learning_rate=lea_rat)
train_loss_results = []
test_loss_results = []
num_epochs = num_epo
train_acc_metric = tf.keras.metrics.MeanSquaredError(name="mse")
test_acc_metric = tf.keras.metrics.MeanSquaredError(name="mse")

# learning_function = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=lea_rat,
#     decay_steps=num_epo
# )

@tf.function
@tf.autograph.experimental.do_not_convert
def train_step(model, x, y):
    with tf.GradientTape() as tape:
        loss_value = tf.reduce_mean(tf.keras.losses.mse(model(x, training=True), y))
    grads = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(model(x, training=True), y)
    return loss_value

@tf.function
@tf.autograph.experimental.do_not_convert
def test_step(model, x, y):
    loss_value = tf.reduce_mean(tf.keras.losses.mse(model(x, training=False), y))
    val_acc_metric.update_state(model(x, training=False), y)
    return loss_value

# training loop
for epoch in range(num_epochs+1):
    epoch_loss_avg = tf.keras.metrics.Mean()
    
    # loop over batches
    for x, y in dat_tra:
        train_loss_value = train_step(model, x, y)
        # track progress
        epoch_loss_avg.update_state(train_loss_value)
    
#     train_loss_value = train_step(model, X_train, y_train)
#     # track progress
#     epoch_loss_avg.update_state(train_loss_value)
    
    # end epoch
    train_loss_results.append(epoch_loss_avg.result())

    if epoch % 50 == 0:
        print("Epoch {:03d}: Loss: {:.6f}".format(epoch, epoch_loss_avg.result()))

(None, 1)
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_25 (Sl (None, 1)            0           input_8[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_22 (Sl (None, 1)            0           input_8[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_26 (Sl (None, 1)            0           input_8[0][0]                    
__________________________________________________________________________________

Epoch 000: Loss: 6.254594
Epoch 050: Loss: 4.532739
Epoch 100: Loss: 5.673643


KeyboardInterrupt: 

In [ ]:
# 2d plot
plt.figure()
ax = plt.gca()
im = ax.imshow(
    np.abs(T_pred[X.shape[0]//2,:,:]-T_data[X.shape[0]//2,:,:]), 
    extent=[np.min(x_plot),np.max(x_plot),np.max(z_plot),np.min(z_plot)], 
    aspect=1,     
    cmap="jet"
)
plt.xlabel('Z (x10^3 km)')
plt.ylabel('X (x10^3 km)')
plt.title('Absolute Travel Time Reisudals')
ax.xaxis.set_major_locator(plt.MultipleLocator(2))
ax.yaxis.set_major_locator(plt.MultipleLocator(2))
ax.plot(sz*ear_rad/1000,sx*ear_rad/1000,'w*',markersize=8)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="6%", pad=0.15)

cbar = plt.colorbar(im, cax=cax, format='%.3e')
cbar.set_label('s',size=10)
cbar.ax.tick_params(labelsize=10)
# cbar.mappable.set_clim(np.min(T_fmm_all), np.max(T_fmm_all[np.isfinite(T_fmm_all)]))
plt.show()

In [ ]:
# 2d plot
plt.figure()
ax = plt.gca()
im = ax.imshow(
    T_data[X.shape[0]//2,:,:], 
    extent=[np.min(x_plot),np.max(x_plot),np.max(z_plot),np.min(z_plot)], 
    aspect=1,     
    cmap="jet"
)
plt.xlabel('Z (x10^3 km)')
plt.ylabel('X (x10^3 km)')
plt.title('Analytical Travel Time')
ax.xaxis.set_major_locator(plt.MultipleLocator(2))
ax.yaxis.set_major_locator(plt.MultipleLocator(2))
ax.plot(sz*ear_rad/1000,sx*ear_rad/1000,'w*',markersize=8)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="6%", pad=0.15)

cbar = plt.colorbar(im, cax=cax, format='%.3e')
cbar.set_label('s',size=10)
cbar.ax.tick_params(labelsize=10)
# cbar.mappable.set_clim(np.min(T_fmm_all), np.max(T_fmm_all[np.isfinite(T_fmm_all)]))
plt.show()

In [ ]:
# 2d plot
plt.figure()
ax = plt.gca()
im = ax.imshow(
    T_pred[X.shape[0]//2,:,:], 
    extent=[np.min(x_plot),np.max(x_plot),np.max(z_plot),np.min(z_plot)], 
    aspect=1,     
    cmap="jet"
)
plt.xlabel('Z (x10^3 km)')
plt.ylabel('X (x10^3 km)')
plt.title('Predicted Travel Time')
ax.xaxis.set_major_locator(plt.MultipleLocator(2))
ax.yaxis.set_major_locator(plt.MultipleLocator(2))
ax.plot(sz*ear_rad/1000,sx*ear_rad/1000,'w*',markersize=8)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="6%", pad=0.15)

cbar = plt.colorbar(im, cax=cax, format='%.3e')
cbar.set_label('s',size=10)
cbar.ax.tick_params(labelsize=10)
# cbar.mappable.set_clim(np.min(T_fmm_all), np.max(T_fmm_all[np.isfinite(T_fmm_all)]))
plt.show()

In [ ]:
# 2d plot
plt.figure()
ax = plt.gca()
im = ax.imshow(
    np.abs(V_pred[X.shape[0]//2,:,:]-vel_mod[X.shape[0]//2,:,:]), 
    extent=[np.min(x_plot),np.max(x_plot),np.max(z_plot),np.min(z_plot)], 
    aspect=1,     
    cmap="jet"
)
plt.xlabel('Z (x10^3 km)')
plt.ylabel('X (x10^3 km)')
plt.title('Absolute Velocity Resiudals')
ax.xaxis.set_major_locator(plt.MultipleLocator(2))
ax.yaxis.set_major_locator(plt.MultipleLocator(2))
ax.plot(sz*ear_rad/1000,sx*ear_rad/1000,'w*',markersize=8)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="6%", pad=0.15)

cbar = plt.colorbar(im, cax=cax, format='%.3e')
cbar.set_label('km/s',size=10)
cbar.ax.tick_params(labelsize=10)
# cbar.mappable.set_clim(2, 3)
plt.show()

In [ ]:
# 2d
plt.figure()
ax = plt.gca()
im = ax.pcolormesh(
    xx[:,latitude.shape[0]//2,:],
    yy[:,latitude.shape[0]//2,:],
    V_pred_gap[::-1,latitude.shape[0]//2,:],
    cmap=plt.get_cmap("jet"),
    shading="gouraud"
)
ax.set_aspect('equal')
plt.xlabel('Z (x10^3 km)')
plt.ylabel('X (x10^3 km)')
plt.title('Predicted Velocity (Gap)')
ax.plot(xx_s,yy_s,'w*',markersize=8)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="6%", pad=0.15)
ax.xaxis.set_major_locator(plt.MultipleLocator(2))

cbar = plt.colorbar(im, cax=cax, format='%.3e')
cbar.set_label('km/s',size=10)
cbar.mappable.set_clim(v_min, v_max)
cbar.ax.tick_params(labelsize=10)
plt.savefig(figures_path + 'V_PINNs_Gap_2d.png', bbox_inches="tight")
# plt.show()

In [ ]:
# 2d
plt.figure()
ax = plt.gca()
im = ax.pcolormesh(
    xx[:,latitude.shape[0]//2,:],
    yy[:,latitude.shape[0]//2,:],
    V_pred[::-1,latitude.shape[0]//2,:],
    cmap=plt.get_cmap("jet"),
    shading="gouraud"
)
ax.set_aspect('equal')
plt.xlabel('Z (x10^3 km)')
plt.ylabel('X (x10^3 km)')
plt.title('Predicted Velocity (All)')
ax.plot(xx_s,yy_s,'w*',markersize=8)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="6%", pad=0.15)
ax.xaxis.set_major_locator(plt.MultipleLocator(2))

cbar = plt.colorbar(im, cax=cax, format='%.3e')
cbar.set_label('km/s',size=10)
cbar.mappable.set_clim(v_min, v_max)
cbar.ax.tick_params(labelsize=10)
plt.savefig(figures_path + 'V_PINNs_all_2d.png', bbox_inches="tight")
# plt.show()

In [ ]:
plt.figure()
ax = plt.gca()
ax.pcolormesh(
    xx[:,latitude.shape[0]//2,:],
    yy[:,latitude.shape[0]//2,:],
    vel_all[::-1,latitude.shape[0]//2,:],
    cmap=plt.get_cmap("jet"),
    shading="gouraud"
)
ax.set_aspect('equal')
plt.xlabel('Z (x10^3 km)')
plt.ylabel('X (x10^3 km)')
plt.title('Initial Velocity (All)')
ax.plot(xx_s,yy_s,'w*',markersize=8)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="6%", pad=0.15)
ax.xaxis.set_major_locator(plt.MultipleLocator(2))

cbar = plt.colorbar(im, cax=cax, format='%.3e')
cbar.set_label('km/s',size=10)
cbar.mappable.set_clim(v_min, v_max)
cbar.ax.tick_params(labelsize=10)
plt.savefig(figures_path + 'V_ini_all_2d.png', bbox_inches="tight")
# plt.show()

In [ ]:
# 2d
plt.figure()
ax = plt.gca()
im = ax.pcolormesh(
    xx[:,latitude.shape[0]//2,:],
    yy[:,latitude.shape[0]//2,:],
    vel_gap[::-1,latitude.shape[0]//2,:],
    cmap=plt.get_cmap("jet"),
    shading="gouraud"
)
ax.set_aspect('equal')
plt.xlabel('Z (x10^3 km)')
plt.ylabel('X (x10^3 km)')
plt.title('Initial Velocity (Gap)')
ax.plot(xx_s,yy_s,'w*',markersize=8)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="6%", pad=0.15)
ax.xaxis.set_major_locator(plt.MultipleLocator(2))

cbar = plt.colorbar(im, cax=cax, format='%.3e')
cbar.set_label('km/s',size=10)
cbar.mappable.set_clim(v_min, v_max)
cbar.ax.tick_params(labelsize=10)
plt.savefig(figures_path + 'V_ini_gap_2d.png', bbox_inches="tight")
plt.show()

In [ ]:
_, DEP, _ = np.meshgrid(latitude, depth, longitude)

xx = DEP * np.sin(np.radians(LAT+90)) * np.cos(np.radians(180+LON))/(1e3)
yy = DEP * np.sin(np.radians(LAT+90)) * np.sin(np.radians(180+LON))/(1e3)

xx_s = (ear_rad - dep_sou) * np.sin(np.radians(lat_sou+90)) * np.cos(np.radians(180+lon_sou))/(1e3)
yy_s = (ear_rad - dep_sou) * np.sin(np.radians(lat_sou+90)) * np.sin(np.radians(180+lon_sou))/(1e3)

v_min, v_max = np.min(vel_all), np.max(vel_all)


V_pred_gap = np.where(DEP<3000, V_pred, np.nan)
T_pred_gap = np.where(DEP<3000, T_pred, np.nan)

# plots

In [ ]:
tau_pred = tau.predict(Xp, batch_size=64).reshape((dep_dim, lat_dim, lon_dim))
V_pred = model.predict(Xp, batch_size=64).reshape((dep_dim, lat_dim, lon_dim))
T0 = tf.math.sqrt((X-sx)**2 + (Y-sy)**2 + (Z-sz)**2)
T_pred = tau_pred * T0

# test

In [ ]:
tau_pred = tau.predict(Xp, batch_size=64).reshape((nx, ny, nz))
V_pred = model.predict(Xp, batch_size=64).reshape((nx, ny, nz))
T0 = tf.math.sqrt((X-sx)**2 + (Y-sy)**2 + (Z-sz)**2)
T_pred = tau_pred * T0

In [ ]:
# 2d
plt.figure()
ax = plt.gca()
im = plt.imshow(np.abs(V_pred[:,20,:]-vel_mod[:,20,:]))
ax.set_aspect('equal')
plt.xlabel('Z (x10^3 km)')
plt.ylabel('X (x10^3 km)')
plt.title('Absolute Velocity Residuals')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="6%", pad=0.15)
ax.xaxis.set_major_locator(plt.MultipleLocator(2))

cbar = plt.colorbar(im, cax=cax, format='%.3e')
cbar.set_label('km/s',size=10)
# cbar.mappable.set_clim(2, 3)
cbar.ax.tick_params(labelsize=10)
plt.show()

In [ ]:
tf.keras.optimizers.schedules.PiecewiseConstantDecay?